# Unsupervised examples (Clustering + Reduction)

This notebook provides interactive examples to run and debug the project's pipelines and to execute quick ad-hoc unsupervised analyses (KMeans, DBSCAN, Hierarchical, PCA, t-SNE). It is written to run from the project root. Adjust paths if you open it from a different folder.

In [2]:
# 1) Configurar entorno y dependencias

import os
import sys
import importlib
import traceback
from pathlib import Path
# -- Diagnostic prints: which Python/kernel is executing this notebook?
print('Notebook Python executable:', sys.executable)
print('First entries in sys.path:', sys.path[:6])
# Check if kedro is visible to this interpreter
import subprocess
try:
    res = subprocess.run([sys.executable, '-m', 'pip', 'show', 'kedro'], capture_output=True, text=True)
    print('pip show kedro output:
except Exception as exc:
    print('Failed to run pip show kedro:', exc)

# Detectar automáticamente la raíz del proyecto: busca un padre que contenga 'src', 'pyproject.toml' o 'conf'
def find_project_root(start=Path.cwd()):
    cur = Path(start).resolve()
    for parent in [cur] + list(cur.parents):
        if (parent / 'src').is_dir() or (parent / 'pyproject.toml').is_file() or (parent / 'conf').is_dir():
            return parent
    # Si no encuentra nada, usa el directorio actual (útil si ejecutas desde la raíz manualmente)
    return Path.cwd()
ROOT = find_project_root()
print("Project root (detected):", ROOT)

# Añadir src al path para poder importar el paquete covid19df
sys.path.insert(0, str(ROOT / "src"))

# Rutas de entrada/salida
INPUT_CSV = str(ROOT / "data" / "03_intermediate" / "covid_19_clean_complete_CLEAN.csv")
OUT_DIR = str(ROOT / "data" / "05_train")
os.makedirs(OUT_DIR, exist_ok=True)

# Archivos de salida recomendados
ELBOW_CSV = os.path.join(OUT_DIR, "elbow.csv")
ELBOW_PLOT = os.path.join(OUT_DIR, "elbow.png")
KMEANS_CSV = os.path.join(OUT_DIR, "kmeans_clusters.csv")
KMEANS_METRICS = os.path.join(OUT_DIR, "kmeans_metrics.csv")
DBSCAN_CSV = os.path.join(OUT_DIR, "dbscan_clusters.csv")










print("Input CSV exists:", os.path.exists(INPUT_CSV))TSNE_SCATTER = os.path.join(OUT_DIR, "tsne_scatter.png")PCA_SCATTER = os.path.join(OUT_DIR, "pca_scatter.png")TSNE_EMB = os.path.join(OUT_DIR, "tsne_embeddings.csv")PCA_EMB = os.path.join(OUT_DIR, "pca_embeddings.csv")DENDRO_PLOT = os.path.join(OUT_DIR, "dendrogram.png")HIER_METRICS = os.path.join(OUT_DIR, "hier_metrics.csv")HIER_CSV = os.path.join(OUT_DIR, "hier_clusters.csv")DBSCAN_METRICS = os.path.join(OUT_DIR, "dbscan_metrics.csv")print("Input CSV exists:", os.path.exists(INPUT_CSV))

SyntaxError: unterminated string literal (detected at line 15) (4028283207.py, line 15)

In [2]:
# 2) Inicializar contexto Kedro desde el notebook (ejemplo)

try:
    from kedro.framework.session import KedroSession
    from kedro.config import ConfigLoader
    print("Kedro imports OK")
except Exception as exc:
    print("Kedro no disponible en este entorno:", exc)
    # No fallamos; muchas acciones del notebook se hacen directamente con las funciones

# Ejemplo de cómo crear una sesión (comentar si no quieres ejecutar dentro del notebook)
# from kedro.framework.session import KedroSession
# with KedroSession.create() as session:
#     context = session.load_context()
#     print(type(context))


Kedro no disponible en este entorno: No module named 'kedro'


In [ ]:
# 3) Cargar y listar pipelines registrados

try:
    from covid19df import pipeline_registry as pr
    pipelines = pr.register_pipelines()
    print("Registered pipelines:", list(pipelines.keys()))
    cur = Path.cwd().resolve()
    for parent in [cur] + list(cur.parents):
        if (parent / 'src').is_dir():
            p = str((parent / 'src').resolve())
            if p not in sys.path:
                sys.path.insert(0, p)
            return True
    return False

try:
    from covid19df import pipeline_registry as pr
    pipelines = pr.register_pipelines()
    print("Registered pipelines:", list(pipelines.keys()))
    # Mostrar número de nodos en cada pipeline si tiene el método 'nodes' o similar
    for name, p in pipelines.items():
        try:
            nodes = getattr(p, 'nodes', None)
            if nodes is not None:
                print(f"- {name}: {len(nodes)} nodes")
        except Exception:
            pass
except Exception:
    print("Error al importar/ejecutar register_pipelines (first attempt):")
    import traceback
    traceback.print_exc()
    # Intentar corregir sys.path y reintentar una vez
    if ensure_src_on_path():
        try:
            from covid19df import pipeline_registry as pr
            pipelines = pr.register_pipelines()
            print("Registered pipelines after fixing sys.path:", list(pipelines.keys()))
        except Exception:
            print("Retry failed. Traceback:")
            import traceback
            traceback.print_exc()


Error al importar/ejecutar register_pipelines:


Traceback (most recent call last):
  File "C:\Users\hansi\AppData\Local\Temp\ipykernel_9912\106485718.py", line 4, in <module>
    from covid19df import pipeline_registry as pr
  File "C:\Users\hansi\Desktop\kedro\covid19DF_Ev02\covid19df\src\covid19df\pipeline_registry.py", line 1, in <module>
    from covid19df.pipelines import eda as eda_pipeline
  File "C:\Users\hansi\Desktop\kedro\covid19DF_Ev02\covid19df\src\covid19df\pipelines\eda\__init__.py", line 6, in <module>
    from .pipeline import create_pipeline
  File "C:\Users\hansi\Desktop\kedro\covid19DF_Ev02\covid19df\src\covid19df\pipelines\eda\pipeline.py", line 7, in <module>
    from kedro.pipeline import Pipeline, node
ModuleNotFoundError: No module named 'kedro'


In [4]:
# 4) Ejecutar pipeline 'eda' (modo interactivo) — ejemplo con KedroSession (si está disponible)

try:
    from kedro.framework.session import KedroSession
    print('Using KedroSession to run pipeline "eda" (this may take some time)')
    with KedroSession.create() as session:
        session.run(pipeline_name='eda')
    print('Finished running pipeline eda')
except Exception as exc:
    print('No se pudo ejecutar KedroSession.run - puedes ejecutar Kedro desde terminal:')
    print('  kedro run --pipeline eda')
    traceback.print_exc()


No se pudo ejecutar KedroSession.run - puedes ejecutar Kedro desde terminal:
  kedro run --pipeline eda


Traceback (most recent call last):
  File "C:\Users\hansi\AppData\Local\Temp\ipykernel_9912\2877589214.py", line 4, in <module>
    from kedro.framework.session import KedroSession
ModuleNotFoundError: No module named 'kedro'


In [5]:
# 5) Ejecutar pipelines 'regresion' y 'clasificacion' — ejemplo

try:
    from kedro.framework.session import KedroSession
    print('Running regresion and clasificacion via KedroSession')
    with KedroSession.create() as session:
        session.run(pipeline_name='regresion')
        session.run(pipeline_name='clasificacion')
    print('Finished running regresion and clasificacion')
except Exception:
    print('Si KedroSession falla, ejecuta en terminal:')
    print('  kedro run --pipeline regresion')
    print('  kedro run --pipeline clasificacion')
    traceback.print_exc()


Si KedroSession falla, ejecuta en terminal:
  kedro run --pipeline regresion
  kedro run --pipeline clasificacion


Traceback (most recent call last):
  File "C:\Users\hansi\AppData\Local\Temp\ipykernel_9912\2424806158.py", line 4, in <module>
    from kedro.framework.session import KedroSession
ModuleNotFoundError: No module named 'kedro'


In [ ]:
# 6) Manejar import errors y mostrar traceback (ejemplo de import perezoso)

try:
    # Forzar una importación perezosa similar a pipeline_registry
    import importlib
    importlib.import_module('covid19df.pipelines.unsupervised.clustering.pipeline')
    print('Clustering pipeline module imported OK')
except Exception:
    print('Import error detected: show full traceback')
    traceback.print_exc()
    print('\nRevisa los errores anteriores, suelen implicar dependencias faltantes o errores de sintaxis en los módulos.')


In [ ]:
# 7) Guardar y visualizar artefactos/resultados (ejemplo ad-hoc ejecutando funciones directamente)

# Importar funciones de los nodos que creamos para ejecución ad-hoc
from covid19df.pipelines.unsupervised.clustering.nodes import (
    load_latest_by_country, preprocess_for_clustering, elbow_method,
    run_kmeans, run_dbscan, run_hierarchical, compute_clustering_metrics,
    save_cluster_results, plot_dendrogram, save_metrics_csv
)
from covid19df.pipelines.unsupervised.reduction.nodes import run_pca, run_tsne, save_embeddings

# Cargar y preprocesar
raw = load_latest_by_country(INPUT_CSV)
X_scaled_df, raw_df = preprocess_for_clustering(raw)
print('Data loaded:', raw_df.shape, 'Features:', X_scaled_df.shape)

# Elbow
elbow_res = elbow_method(X_scaled_df, k_min=2, k_max=8, out_csv=ELBOW_CSV, out_plot=ELBOW_PLOT)
print('Elbow saved to', ELBOW_CSV, ELBOW_PLOT)

# KMeans
k = 4
k_labels, k_model = run_kmeans(X_scaled_df, n_clusters=k)
save_cluster_results(raw_df, k_labels, KMEANS_CSV)
k_metrics = compute_clustering_metrics(X_scaled_df, k_labels)
save_metrics_csv(k_metrics, KMEANS_METRICS)
print('KMeans saved:', KMEANS_CSV, KMEANS_METRICS)

# DBSCAN
db_labels, db_model = run_dbscan(X_scaled_df, eps=0.5, min_samples=5)
save_cluster_results(raw_df, db_labels, DBSCAN_CSV)
db_metrics = compute_clustering_metrics(X_scaled_df, db_labels)
save_metrics_csv(db_metrics, DBSCAN_METRICS)
print('DBSCAN saved:', DBSCAN_CSV, DBSCAN_METRICS)

# Hierarchical
h_labels, h_model = run_hierarchical(X_scaled_df, n_clusters=k)
save_cluster_results(raw_df, h_labels, HIER_CSV)
h_metrics = compute_clustering_metrics(X_scaled_df, h_labels)
save_metrics_csv(h_metrics, HIER_METRICS)
print('Hierarchical saved:', HIER_CSV, HIER_METRICS)

# Dendrogram
dendro = plot_dendrogram(X_scaled_df, method='ward', out_plot=DENDRO_PLOT)
print('Dendrogram saved:', DENDRO_PLOT if dendro else 'not generated')

# PCA + t-SNE
pca_res = run_pca(X_scaled_df, n_components=2)
import numpy as np
save_embeddings(raw_df, np.array(pca_res['embeddings']), PCA_EMB)
print('PCA embeddings saved:', PCA_EMB)

try:
    tsne_res = run_tsne(X_scaled_df, n_components=2, perplexity=30.0)
    save_embeddings(raw_df, np.array(tsne_res['embeddings']), TSNE_EMB)
    print('t-SNE embeddings saved:', TSNE_EMB)
except Exception:
    print('t-SNE failed (can be slow); skip tsne embedding')

# Visualizar scatter de PCA con etiquetas KMeans (si matplotlib disponible)
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    emb = np.array(pca_res['embeddings'])
    plt.figure(figsize=(6,5))
    sns.scatterplot(x=emb[:,0], y=emb[:,1], hue=k_labels, palette='tab10', s=40)
    plt.title('PCA scatter colored by KMeans')
    plt.tight_layout()
    plt.savefig(PCA_SCATTER)
    plt.close()
    print('PCA scatter saved:', PCA_SCATTER)
except Exception:
    print('matplotlib/seaborn not available; skipping scatter plot')

try:
    if os.path.exists(TSNE_EMB):
        emb2 = np.loadtxt(TSNE_EMB, delimiter=',', skiprows=1)[:,1:3] if False else None
except Exception:
    pass


In [ ]:
# 8) Ejecutar tests unitarios desde el notebook (ejemplo)

# Puedes ejecutar pytest desde el notebook usando una celda bash o subprocess
print('Running pytest -k eda (example). This runs in a subprocess and prints output below:')

import subprocess
try:
    res = subprocess.run([sys.executable, '-m', 'pytest', '-q'], capture_output=True, text=True)
    print(res.stdout)
    if res.returncode != 0:
        print('Some tests failed. See above output.')
except Exception as exc:
    print('Unable to run pytest from notebook:', exc)


In [ ]:
# 9) Reproducibilidad: fijar seed y mostrar versiones

import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Mostrar versiones
import platform
import pkgutil
print('Python:', platform.python_version())
for pkg in ('kedro','pandas','numpy','scikit_learn','matplotlib','scipy','seaborn'):
    try:
        loader = pkgutil.find_loader(pkg)
        print(pkg, 'installed:', loader is not None)
    except Exception:
        print(pkg, 'check failed')

print('\nDone. The notebook saved outputs to', OUT_DIR)
